In [ ]:
'''
General Neuron:
   |______Sensory Neuron:
   |            |____Photoreceptor
   |            |____Mechanoreceptor
   |
   |______Motor Neuron:
                |____Alpha Motor Neuron
                |____Gamma Motor Neuron
'''

In [2]:
pip install numpy matplotlib panda

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7244 sha256=e30a51af6de04fb58b49f3c51835d9b32261eb6c5ef65ce6be57200e89e675a7
  Stored in directory: c:\users\matan\appdata\local\pip\cache\wheels\53\7c\f7\d3267739bc9dfdfc43724391337c9955e7d5b5830888d735e3
Successfully built panda
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# A generic neuron with basic properties and functions common to all neuron types.
class GeneralNeuron:
    # Constructor (with defualt value for firing_rate)
    def __init__(self, firing_rate=0.0):
        self.firing_rate = firing_rate

    # Calculates a firing rate based on the strength of that stimulus.
    def Activate(self, stimulus):
        pass

# A neuron that detects and responds to external stimuli. Each sensory neuron is sensitive to a specific type of stimulus.
class SensoryNeuron(GeneralNeuron):
    # Constructor
    def __init__(self, firing_rate, receptor_type):
        super().__init__(firing_rate)
        self.receptor_type = receptor_type

# Process the specific stimulus and activate the neuron
    def SenseStimulus(self):
        pass

# A neuron responsible for initiating muscle movement by sending activation signals to a target muscle.
class MotorNeuron(GeneralNeuron):
    # Constructor
    def __init__(self, firing_rate, target_muscle):
        super().__init__(firing_rate)
        self.target_muscle = target_muscle

    # Trigger a response in the target muscle
    def ControlMuscle(self):
        pass

# A sensory neuron specifically sensitive to light.
class Photoreceptor(SensoryNeuron):
    # Constructor
    def __init__(self, firing_rate):
        super().__init__(firing_rate, receptor_type="light")

    # Activate based on light intensity
    def LightDetection(self, light_intensity):
        pass

# A sensory neuron that responds to pressure.
class Mechanoreceptor(SensoryNeuron):
    # Constructor
    def __init__(self, firing_rate):
        super().__init__(firing_rate, receptor_type="pressure")

    # Activate based on pressure level
    def PressureDetection(self, pressure):
        pass

# A motor neuron that controls skeletal muscles.
class AlphaMotorNeuron(MotorNeuron):
    # Constructor
    def __init__(self, firing_rate):
        super().__init__(firing_rate, target_muscle="skeletal muscle")

    # Initiate muscle contraction or movement
    def SkeletalMuscleControl(self):
        pass

# A motor neuron that controls muscle spindles, helping regulate muscle tone.
class GammaMotorNeuron(MotorNeuron):
    # Constructor
    def __init__(self, firing_rate):
        super().__init__(firing_rate, target_muscle="muscle spindle")

    # Adjust muscle spindle tension
    def MuscleSpindleControl(self):
        pass